# Mixture of Experts (MoE)

- MoE의 핵심은 transformer block에 있는 FFN을 여러개의 전문가 layer (multiple expert layer)로 교체하는 것.
  - 물론 이 expert layer 하나하나도 FFN module 임.
- 즉, **single feed-forward block을 여러개의 feed-forward block으로 교체**하는 것.


![MoE](https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/moe-memory/1.webp)


- 일반적으로 Transformer block 내부에 있는 FFN은 모델의 전체 파라미터 수 대부분을 차지함.
  - DeepSeek-V3의 경우 feed-forward block이 61개 있음.
- 즉, 단일 feed-forward block을 여러개의 block으로 교체하면(MoE), 모델의 전체 파라미터 수가 상당히 증가함.
- 하지만 핵심은, **모든 token에 대해 모든 expert를 사용하는 것이 아니라, router를 통해 token당 소수의 expert만 선택**한다는 점.
- **동시에 활성화 되는 expert가 소수**에 불과하기 때문에, **MoE module은 전체 파라미터 set을 항상 사용하는 dense module(FFN)과 반대로 sparse module**로 여겨지는 경우가 많음.
- 하지만 MoE를 통해 전달되는 파라미터의 총 개수가 많아지면 LLM의 capacity가 증가해, training 과정에서 더 많은 지식을 습득할 수 있음.
- 또한 **모든 파라미터를 동시에 사용하지 않는 sparsity 덕분에 inference가 efficient** 해지게 됨.
  - 예시로, DeepSeek-V3는 MoE module 당 256개의 expert와 총 671 billion(6710억)개의 파라미터를 보유하고 있지만, inference 시에는 한번에 9개의 expert만 활성화 됨. (1개의 shared expert + router가 선택한 8개의 expert)
  - 즉, **token inference step마다 671 billion개의 파라미터가 아닌, 단 37 billion(370억)개의 파라미터만 사용**되는 것.
  - DeepSeek-V3의 MoE 설계에서의 특징 중 하나는 **shared expert**를 사용하는 점인데, 이 expert는 모든 token에 대해 항상 활성화 되어 있음. ([2022 DeepSpeed-MoE](https://arxiv.org/abs/2201.05596) 과 [2024 DeepSeek MoE](https://arxiv.org/abs/2401.06066)에서 먼저 선보였던 아이디어임.)


![MoE_shared expert](https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/moe-memory/3.webp)
([DeepSeekMoE: Towards Ultimate Expert Specialization in Mixture-of-Experts Language Models](https://arxiv.org/abs/2401.06066) paper에서 언급된 figure.)

- 이처럼 expert를 공유하는 것의 장점은 [DeepSpeed-MoE](https://arxiv.org/abs/2201.05596)에서 처음 언급되었는데, 이 논문에선 expert를 공유하지 않는 경우에 비해 전반적인 modeling 성능이 향상된다는 점을 발견함.
  - 공통적이거나 반복되는 패턴은 여러 expert가 개별적으로 학습할 필요가 없기 때문에, expert들이 specialized pattern을 학습하는데 더 많은 여유를 가질 수 있으므로 성능이 향상될 가능성이 높아지는 것.

### Code Examples

- GPT-2는 전통적으로 GELU를 사용하지만, 여기선 [SwiGLU](https://arxiv.org/abs/2002.05202)를 사용함.
- 마찬가지로 trained model이 아니므로 의미없는 텍스트를 생성함.
  - Trained는 ch05 - qwen3-moe-plus-kvcache 를 참고.

In [1]:
import time
import tiktoken
import torch
import torch.nn as nn

MOE_FF_TIME_MS = []
MOE_FF_MEM_BYTES = []

In [2]:
from _04_gpt_with_kv_cache import MultiHeadAttention, LayerNorm, FeedForward

class MoEFeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.num_experts_per_token = cfg["num_experts_per_token"]
        self.num_experts = cfg["num_experts"]
        self.embed_dim = cfg["embed_dim"]

        # Gating network
        self.gate = nn.Linear(cfg["embed_dim"], cfg["num_experts"], bias=False)

        # Experts
        self.fc1 = nn.ModuleList([
            nn.Linear(cfg["embed_dim"], cfg["hidden_dim"], bias=False)
            for _ in range(self.num_experts)
        ])
        self.fc2 = nn.ModuleList([
            nn.Linear(cfg["hidden_dim"], cfg["embed_dim"], bias=False)
            for _ in range(self.num_experts)
        ])
        self.fc3 = nn.ModuleList([
            nn.Linear(cfg["hidden_dim"], cfg["embed_dim"], bias=False)
            for _ in range(self.num_experts)
        ])
    
    def forward(self, x):
        scores = self.gate(x)   # [B, seq_len, num_experts]

        # 각 token마다 top-k expert 선택
        topk_scores, topk_indices = torch.topk(scores, self.num_experts_per_token, dim=-1)
        topk_probs = torch.softmax(topk_scores, dim=-1)

        batch, seq_len, _ = x.shape

        # 좀 더 쉬운 indexing을 위해 2D로 변환
        x_flat = x.reshape(batch * seq_len, -1) # [B * seq_len, embed_dim]

        # output tensor 초기화
        out_flat = torch.zeros(batch * seq_len, self.embed_dim, device=x.device, dtype=x.dtype)

        # 마찬가지로 x_flat과 차원을 맞추기 위해 top-k indices, probs를 2D로 변환
        topk_indices_flat = topk_indices.reshape(-1, self.num_experts_per_token)
        topk_probs_flat = topk_probs.reshape(-1, self.num_experts_per_token)

        # 모든 선택된 expert들의 unique한 집합 구하기
        unique_experts = torch.unique(topk_indices_flat)

        for expert_id_tensor in unique_experts:
            expert_id = int(expert_id_tensor.item())

            # 이 expert가 선택된 token들의 마스크 생성
            mask = topk_indices_flat == expert_id  # [B * seq_len, num_experts_per_token]
            if not mask.any():
                continue

            # 선택된 token들만 추출
            token_mask = mask.any(dim=-1)
            selected_idx = token_mask.nonzero(as_tuple=False).squeeze(-1)
            if selected_idx.numel() == 0:
                continue

            # 해당 expert로 포워드 패스
            expert_input = x_flat.index_select(0, selected_idx)
            hidden = torch.nn.functional.silu(self.fc1[expert_id](expert_input)) * self.fc2[expert_id](expert_input)
            expert_out = self.fc3[expert_id](hidden)

            # 확률로 가중치 적용
            mask_selected = mask[selected_idx]
            slot_indicies = mask_selected.int().argmax(dim=-1, keepdim=True)
            selected_probs = torch.gather(
                topk_probs_flat.index_select(0, selected_idx),
                dim=-1,
                index=slot_indicies
            ).squeeze(-1)

            # 결과를 원래 위치에 더하기
            out_flat.index_add_(0, selected_idx, expert_out * selected_probs.unsqueeze(-1))
            final_output = out_flat.reshape(batch, seq_len, self.embed_dim)
        
        return final_output

In [4]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.attention = MultiHeadAttention(
            d_in = cfg['embed_dim'],
            d_out = cfg['embed_dim'],
            context_length = cfg['context_length'],
            num_heads = cfg['num_heads'],
            dropout = cfg['drop_rate'],
            qkv_bias = cfg['qkv_bias']
        )

        # expert가 명시되어 있으면 MoE 사용, 그렇지 않으면 FFN
        self.ffn = MoEFeedForward(cfg) if cfg["num_experts"] > 0 else FeedForward(cfg)

        self.norm1 = LayerNorm(cfg['embed_dim'])
        self.norm2 = LayerNorm(cfg['embed_dim'])

        self.drop_shortcut = nn.Dropout(cfg['drop_rate'])

    def forward(self, x, use_cache=False):
        """
        input -> LayerNorm -> MHA -> Dropout -> skip connection
        -> LayerNorm -> FFN -> Dropout -> skip connection
        -> output
        """
        # attention with skip connection
        residual = x
        x = self.norm1(x)           # LayerNorm
        x = self.attention(x, use_cache=use_cache)       # MHA, [batch, context_length, embed_dim]
        x = self.drop_shortcut(x)   # Dropout
        x = x + residual            # skip(residual) connection

        # FFN with skip connection
        residual = x
        x = self.norm2(x)           # LayerNorm
        x = self.ffn(x)             # FeedForward

        # Memory tracking (optional)
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            torch.cuda.synchronize()
            torch.cuda.reset_peak_memory_stats()
            base_mem = torch.cuda.memory_allocated()
        start = time.perf_counter()

        x = self.ffn(x)             # FeedForward

        if use_cuda:
            torch.cuda.synchronize()
            peak_mem = torch.cuda.max_memory_allocated() - base_mem
            MOE_FF_MEM_BYTES.append(peak_mem - base_mem)
        MOE_FF_TIME_MS.append((time.perf_counter() - start) * 1000)

        x = self.drop_shortcut(x)   # Dropout
        x = x + residual            # skip(residual) connection

        return x

In [5]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.token_embedding = nn.Embedding(cfg['vocab_size'], cfg['embed_dim'])
        self.position_embedding = nn.Embedding(cfg['context_length'], cfg['embed_dim'])
        self.drop_embedding = nn.Dropout(cfg['drop_rate'])

        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(cfg) for _ in range(cfg['num_layers'])]
        )
        self.current_pos = 0

        self.final_norm = LayerNorm(cfg['embed_dim'])
        self.out_head = nn.Linear(cfg['embed_dim'], cfg['vocab_size'], bias=False)

    def forward(self, in_idx, use_cache=False):
        batch_size, seq_length = in_idx.shape

        # token embedding, positional embedding을 더해서 최종 input embedding 구성
        token_embeddings = self.token_embedding(in_idx)
        
        if use_cache:
            pos_ids = torch.arange(
                self.current_pos, self.current_pos + seq_length, device=in_idx.device, dtype=torch.long
            )
            self.current_pos += seq_length
        else:
            pos_ids = torch.arange(
                0, seq_length, device=in_idx.device, dtype=torch.long
            )
        pos_embeddings = self.position_embedding(pos_ids).unsqueeze(0)

        x = token_embeddings + pos_embeddings   # [batch_size, num_tokens, embed_dim]

        x = self.drop_embedding(x)

        # Transformer block forward pass
        for block in self.transformer_blocks:
            x = block(x, use_cache=use_cache)

        # last layer norm
        x = self.final_norm(x)

        logits = self.out_head(x)

        return logits
    
    def reset_kv_cache(self):
        for block in self.transformer_blocks:
            block.attention.reset_cache()
        self.current_pos = 0

In [6]:
def generate_text_simple_cached_with_MoE(model, idx, max_new_tokens, context_size=None, use_cache=True):
    model.eval()
    context_length = context_size or model.position_embedding.num_embeddings

    batch_size, base_len = idx.shape
    total_len = base_len + max_new_tokens
    generated = torch.empty(
        batch_size, total_len, dtype=idx.dtype, device=idx.device
    )
    generated[:, :base_len] = idx

    current_len = base_len
    use_cuda = torch.cuda.is_available()
    MOE_FF_MEM_BYTES.clear()
    MOE_FF_TIME_MS.clear()

    with torch.no_grad():
        if use_cache:
            # Initialize cache with full prompt
            model.reset_kv_cache()
            prompt_start = max(0, current_len - context_length)
            logits = model(idx[:, -context_length:], use_cache=True)

            if use_cuda:
                torch.cuda.synchronize()

            for _ in range(max_new_tokens):
                # 가장 높은 log-probability를 가진 token 선택 (greedy sampling)
                next_idx = logits[:, -1].argmax(dim=-1)

                # 새로운 token을 입력 sequence에 추가
                generated[:, current_len] = next_idx
                current_len += 1

                # model에는 새 token만을 전달
                logits = model(generated[:, current_len-1 : current_len], use_cache=True)

                if use_cuda:
                    torch.cuda.synchronize()
        
        else:
            if use_cuda:
                torch.cuda.synchronize()

            for _ in range(max_new_tokens):
                start_context = max(0, current_len - context_length)
                logits = model(generated[:, start_context:current_len], use_cache=False)
                next_idx = logits[:, -1].argmax(dim=-1, keepdim=True)
                generated[:, current_len] = next_idx
                current_len += 1

                if use_cuda:
                    torch.cuda.synchronize()
    
    if MOE_FF_TIME_MS:
        avg_time = sum(MOE_FF_TIME_MS) / len(MOE_FF_TIME_MS)
        print(f"Average MoE FeedForward Time per call: {avg_time:.3f} ms")

    if MOE_FF_MEM_BYTES:
        avg_mem = sum(MOE_FF_MEM_BYTES) / len(MOE_FF_MEM_BYTES)
        max_ffn_mem = max(MOE_FF_MEM_BYTES)
        print(f"Average MoE FeedForward Peak Memory per call: {avg_mem / (1024 ** 2):.3f} MB (Max: {max_ffn_mem / (1024 ** 2):.3f} MB)")

    return generated[:, :current_len]

In [7]:
start_context = "O say can you see,"

tokenizer = tiktoken.get_encoding("gpt2")
encoded = tokenizer.encode(start_context)

GPT_CONFIG_124M = {
        'vocab_size': 50257,                    # Vocabulary size
        'context_length': 200 + len(encoded),   # Context(max sequence) length
        'embed_dim': 768,                       # Embedding dimension
        'hidden_dim': 768,                    # FeedForward hidden dimension
        'num_heads': 12,                        # Number of attention heads
        'num_layers': 12,                       # Number of layers(transformer blocks)
        'drop_rate': 0.1,                       # Dropout rate
        'qkv_bias': False,                      # Q,K,V bias
        'num_experts': 16,                      # Number of MoE experts
        'num_experts_per_token': 2,             # Number of experts per token
    }

torch.manual_seed(62)
model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

model.to(device, dtype=torch.bfloat16)
model.eval()  # disable dropout

encoded_tensor = torch.tensor(encoded, device=device).unsqueeze(0)
print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
print("\nInput text:", start_context)
print("Encoded input text:", encoded)
print("encoded_tensor.shape:", encoded_tensor.shape)
    
if torch.cuda.is_available():
    torch.cuda.synchronize()
start = time.time()

token_ids = generate_text_simple_cached_with_MoE(
    model = model,
    idx = encoded_tensor,
    max_new_tokens = 200
)

if torch.cuda.is_available():
    torch.cuda.synchronize()
total_time = time.time() - start

decoded_text = tokenizer.decode(token_ids.squeeze(0).tolist())
print(f"\n\n{50*'='}\n{22*' '}OUT\n{50*'='}")
print("\nOutput:", token_ids)
print("Output length:", len(token_ids[0]))
print("Output text:", decoded_text)

print(f"\nTime: {total_time:.2f} sec")
print(f"{int(len(token_ids[0])/total_time)} tokens/sec")

if torch.cuda.is_available():
    max_mem_bytes = torch.cuda.max_memory_allocated()
    max_mem_gb = max_mem_bytes / (1024 ** 3)
    print(f"Max memory allocated: {max_mem_gb:.2f} GB")

Using device: cuda

                      IN

Input text: O say can you see,
Encoded input text: [46, 910, 460, 345, 766, 11]
encoded_tensor.shape: torch.Size([1, 6])
Average MoE FeedForward Time per call: 2.446 ms
Average MoE FeedForward Peak Memory per call: -895.075 MB (Max: -891.307 MB)


                      OUT

Output: tensor([[   46,   910,   460,   345,   766,    11, 40078, 32778, 20287, 41527,
         33766, 40867,  4077, 33438,  7976, 33557,   184, 45358,  6777, 48365,
         20812, 25066, 21589, 48452, 10285, 28860,  3290, 37345, 18932, 16023,
         32219,  7923, 24055, 14621, 45859,  6613, 32027, 32370, 48035, 41387,
         43422, 41653, 24170, 17933, 15709, 48504, 46026, 40867, 49087, 40950,
         39668, 16164,  6544, 26305, 10684, 21417,  2011, 18680, 34249, 20748,
         48858, 47141, 42595,  3706, 40598,   906, 21692, 46662, 11597, 34249,
         10039, 13194,  7261, 12970, 14973,  6076, 19807, 36196,  8419, 29718,
         42442, 41958, 11846, 32352, 29

In [8]:
from _04_gpt_with_kv_cache import main
main()

Using device: cuda

                      IN

Input text: O say can you see,
Encoded input text: [46, 910, 460, 345, 766, 11]
encoded_tensor.shape: torch.Size([1, 6])


                      OUT

Output: tensor([[   46,   910,   460,   345,   766,    11,  2213,  8344, 31206,  5306,
         21397, 33975, 27504, 11423, 48095, 27370, 22947,  5887, 41265, 22451,
         29247, 17442, 19084, 11193, 36335, 47439, 13261,  4605,  6148, 33883,
          5357, 48165, 14897,  2718, 36130, 30210, 35545, 44590, 17908, 25627,
         44931, 18421, 10639, 29587, 35404,  9167, 36746, 45355, 43651,  5747,
         35434, 13811, 19969, 33151, 10422, 28924, 19252, 38586, 13270, 44607,
         21911, 18656, 41031, 33233, 23345, 16265, 46451,  9519, 43042, 15282,
         32318, 10110, 32940, 19949, 39966, 44046,  7773, 10712, 12995,  3246,
            25, 30030, 44784, 40684, 37284,    66, 39577, 25370,  6908, 18202,
         33103,  5048,  6171, 42269, 27385,  1218, 49331, 23697, 36703, 20438,
      

- 전체 메모리 사용량은 MoE가 적은 반면 (sparse 하기 때문), 전체적인 feed-forward module이 늘어나므로 총 compute time은 증가함.
  - 즉, MoE를 사용하면 FFN 메모리 사용량이 훨씬 줄어드는 이점은 있지만, 연산 시간은 더 늘어난다는 단점이 있음.
  - 어떻게 보면 전체적인 memory efficiency를 가지고 갈 것인지, computation efficiency를 가지고 갈 것인지의 trade-off를 하는 듯.